# Compute Nino3.4 DJF index for each model, and save to file

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.signal import detrend
from matplotlib import pyplot as plt
from eofs.xarray import Eof
from scipy import signal
import pandas as pd
import xarray as xr
import xesmf as xe
import pprint 
import intake
import util 

# choose where to load data from:
load_data_from = 'cloud'
#load_data_from = 'glade'

if load_data_from == 'glade':
    col = intake.open_esm_datastore("../catalogs/glade-cmip6.json")
    file = 'available_data.txt'
else:
    col_url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
    col = intake.open_esm_datastore(col_url)
    #col = intake.open_esm_datastore("../catalogs/pangeo-cmip6.json")
    file = 'available_data_cloud.txt'

In [2]:
# pick only models with at least 496 yrs in piControl
minyrs_control = 496;
# models with fewer years often missed future scenarios, so they are not so interesting for us

# load table:
data_table = pd.read_table(file,index_col=0)
models_used = data_table['piControl (yrs)'][data_table['piControl (yrs)'] >= minyrs_control].index
print(models_used)

Index(['BCC-CSM2-MR', 'CanESM5', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'E3SM-1-0',
       'EC-Earth3', 'EC-Earth3-Veg', 'MIROC-ES2L', 'MIROC6', 'HadGEM3-GC31-LL',
       'HadGEM3-GC31-MM', 'UKESM1-0-LL', 'MRI-ESM2-0', 'GISS-E2-1-G', 'CESM2',
       'CESM2-WACCM', 'GFDL-ESM4', 'SAM0-UNICON', 'MCM-UA-1-0'],
      dtype='object')


## Choose what model to use 

In [3]:
model = models_used[14]
model

'CESM2'

In [4]:
data_table.loc[model]

piControl (ens.mem.)          1
historical (ens.mem.)        11
ssp126 (ens.mem.)             1
ssp245 (ens.mem.)             1
ssp370 (ens.mem.)             2
ssp585 (ens.mem.)             2
abrupt-4xCO2 (ens.mem.)       1
piControl (yrs)            1200
historical (yrs)            165
ssp126 (yrs)                 86
ssp245 (yrs)                 86
ssp370 (yrs)                 86
ssp585 (yrs)                 86
abrupt-4xCO2 (yrs)          999
Name: CESM2, dtype: object

In [5]:
# what experiments does this model have that we want to study?
if any(data_table.loc[model][:6] == 'data problem') == False:
    exp_list = [exp[:-11] for exp in data_table.loc[model][:6].index if float(data_table.loc[model][:6][exp]) > 0]
else:
    exp_list = []
    for exp in (data_table.loc[model][:6].index):
        if  (data_table.loc[model][:6][exp] != 'data problem'):
            exp_list = np.append(exp_list, exp[:-11])
print(exp_list)   

['piControl', 'historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']


In [7]:
exp_keys = {}; datasets = {}

for exp in exp_list:
#for exp in [exp_list[1]]:
    print(exp)
    #cat = col.search(experiment_id = exp, source_id = model, variable_id='ts', table_id='Amon', member_id = 'r1i1p1f1')
    cat = col.search(experiment_id = exp, source_id = model, variable_id='ts', table_id='Amon') 
        
    dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})
    for key in dset_dict.keys():
        exp_keys[exp] = key
        datasets[key] = dset_dict[key]

exp_keys

piControl
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
historical
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
ssp126
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
ssp245
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
ssp370
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
ssp585
--> The keys in the returned di

{'piControl': 'CMIP.NCAR.CESM2.piControl.Amon.gn',
 'historical': 'CMIP.NCAR.CESM2.historical.Amon.gn',
 'ssp126': 'ScenarioMIP.NCAR.CESM2.ssp126.Amon.gn',
 'ssp245': 'ScenarioMIP.NCAR.CESM2.ssp245.Amon.gn',
 'ssp370': 'ScenarioMIP.NCAR.CESM2.ssp370.Amon.gn',
 'ssp585': 'ScenarioMIP.NCAR.CESM2.ssp585.Amon.gn'}

In [8]:
# load a dataset for manual calendar check:
exp = exp_list[0]; print(exp)
key = exp_keys[exp]
exp_datasets = datasets[key]
members_sorted = exp_datasets.member_id.sortby(exp_datasets.member_id)

ds = exp_datasets.sel(member_id = members_sorted[0])
#print(ds.time)

# results are stored in this if-test:
if model in ['BCC-CSM2-MR', 'FGOALS-g3', 'CanESM5', 'E3SM-1-0', 'GISS-E2-1-G', 'GISS-E2-1-H', 'CESM2', 'CESM2-WACCM', 'GFDL-CM4', 'SAM0-UNICON', 'GFDL-ESM4', 'MCM-UA-1-0']:
    ds_calendar = 'noleap'
elif model in ['CNRM-CM6-1', 'CNRM-ESM2-1', 'IPSL-CM6A-LR', 'MIROC-ES2L', 'MIROC6']:
    ds_calendar = 'gregorian'
elif model in ['EC-Earth3', 'EC-Earth3-Veg', 'MRI-ESM2-0']:
    ds_calendar = 'proleptic_gregorian'
elif model in ['UKESM1-0-LL', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM']:
    ds_calendar = '360_day'
    
print(ds_calendar, 'calendar')

piControl
noleap calendar


In [9]:
def area_weights(lat_bnds, lon_bnds): 
    # computes exact area weigths assuming earth is a perfect sphere
    lowerlats = np.radians(lat_bnds[:,0]); upperlats = np.radians(lat_bnds[:,1])
    difflon = np.radians(np.diff(lon_bnds[0,:])) # if the differences in longitudes are all the same
    areaweights = difflon*(np.sin(upperlats) - np.sin(lowerlats));
    areaweights /= areaweights.mean()
    return areaweights # list of weights, of same dimension as latitude

# function copied from: http://xarray.pydata.org/en/stable/examples/monthly-means.html
def leap_year(year, calendar='standard'):
    """Determine if year is a leap year"""
    leap = False
    if ((calendar in ['standard', 'gregorian',
        'proleptic_gregorian', 'julian']) and
        (year % 4 == 0)):
        leap = True
        if ((calendar == 'proleptic_gregorian') and
            (year % 100 == 0) and
            (year % 400 != 0)):
            leap = False
        elif ((calendar in ['standard', 'gregorian']) and
                 (year % 100 == 0) and (year % 400 != 0) and
                 (year < 1583)):
            leap = False
    return leap

# function copied from: http://xarray.pydata.org/en/stable/examples/monthly-means.html
def get_dpm(time, calendar='standard'):
    """
    return a array of days per month corresponding to the months provided in `months`
    """
    month_length = np.zeros(len(time), dtype=np.int)

    cal_days = dpm[calendar]

    for i, (month, year) in enumerate(zip(time.month, time.year)):
        month_length[i] = cal_days[month]
        if leap_year(year, calendar=calendar) and month == 2: # the feb-test is missing at the website!
            month_length[i] += 1
    return month_length

In [6]:
# inspiration taken from: http://xarray.pydata.org/en/stable/examples/monthly-means.html

# days per month:
dpm = {'noleap': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'proleptic_gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '360_day': [0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]
      }

def day_weights(ds, chosen_season = 'DJF', calendar = 'noleap'):
    month_length = xr.DataArray(get_dpm((ds.time.to_index()), calendar=ds_calendar), coords=[ds.time], name='month_length')
    if chosen_season == 'DJF':
        season_months = month_length.where(month_length['time.season'] == season)
        # repeat last December month, and move it to the beginning
        season_months = xr.concat([season_months[-1], season_months], dim = 'time')

        norm_by_annual = season_months[1:].groupby('time.year').mean('time') # make annual mean
        norm_by_monthly = np.concatenate([np.tile(norm_by_annual.values[i], 12) for i in range(len(norm_by_annual.values))])
        # repeat last December month to give it equal length as season_months. Value of last month will not be used.
        norm_by_monthly = np.concatenate([norm_by_monthly, [norm_by_monthly[-1]]])

        weights = season_months/norm_by_monthly
        # make weigths have mean 1 in chosen season for all years
        # can be checked by running weights.rolling(min_periods=3, center=True, time=3).mean()
        # note that these weights start with a December month
    elif chosen_season == 'all':
        norm_by_annual = month_length.groupby('time.year').mean('time') # make annual mean
        norm_by_monthly = np.concatenate([np.tile(norm_by_annual.values[i], 12) for i in range(len(norm_by_annual.values))])
        weights = month_length/norm_by_monthly
        # normalized to have mean 1
    # if other season wanted, continue developing this if-test
    
    # NB: normalised weights do not care what numbers are produced for other seasons
    return weights 

In [11]:
latregion = slice(-5,5); lonregion = slice(190, 240) # = 120 W - 170 W
# use larger region before regridding, that adds 5 deg to each border:
larger_latregion = slice(-10,10); larger_lonregion = slice(185, 245)

resolution = 1;
ds_out = xr.Dataset({'lon': (['lon'], np.arange(lonregion.start+resolution/2, lonregion.stop+resolution/2, resolution)),
                     'lat': (['lat'], np.arange(latregion.start+resolution/2, latregion.stop+resolution/2, resolution))
                    }
                   )
    
regr_lat_bnds = np.array([[upper, upper+resolution] for upper in range(latregion.start,latregion.stop)])
regr_lon_bnds = np.array([[upper, upper+resolution] for upper in range(lonregion.start,lonregion.stop)])
area_w = area_weights(regr_lat_bnds, regr_lon_bnds)

season = 'DJF'
lastD = {}

for exp in exp_list:
#for exp in exp_list[:2]:
    key = exp_keys[exp]
    exp_datasets = datasets[key]
    members_sorted = exp_datasets.member_id.sortby(exp_datasets.member_id)
    #for member in [members_sorted.values[0]]: # check for first member only
    for member in members_sorted.values:
        print(exp, member)
        ds = exp_datasets.sel(member_id = member)
        
        # select regional data, perform a regridding, and compute area average
        if model == 'MCM-UA-1-0':
             ds = ds.rename({'longitude': 'lon','latitude': 'lat'}) 
        regional_data = ds.ts.sel(lat = larger_latregion, lon = larger_lonregion)
        regridder = xe.Regridder(regional_data, ds_out, 'bilinear', reuse_weights = True)
        regridded_data = regridder(regional_data)
        area_avg = (regridded_data.transpose('time', 'lon', 'lat') * area_w).mean(dim=['lon', 'lat'])
            
        yrs = int(area_avg.shape[0]/12)
        
        weights = day_weights(area_avg, chosen_season = season, calendar = ds_calendar)
        # double check that weights are 1 for all seasons
        meanweights = weights.rolling(min_periods=3, center=True, time=3).mean()
        print('years in experiment:', yrs, '    ',  'mean weights all 1?', all(meanweights.dropna(dim = 'time') == 1))
        
        if exp == 'historical':
            # save last december month for each member for use in season mean in first year of ssp exps
            lastD[member] = area_avg[-1] 
            weights = weights[1:] # drop first december month
        elif exp == 'piControl':
            weights = weights[1:] # drop first december month
        elif exp not in ['piControl','historical']: # then it must be future scenario   
            area_avg = xr.concat([lastD[member], area_avg], dim = 'time')  
            weights = weights.assign_coords(time = area_avg.time)
            
        # average over season
        day_weighted_avg = area_avg*weights
        ds_season = day_weighted_avg.where(day_weighted_avg['time.season'] == season) # creates nan in all other months
            
        ds_season3 = ds_season.rolling(min_periods=3, center=True, time=3).mean()
        
        if exp not in ['piControl','historical']:
            # remove nan-value obtained from inserting last december month from historical
            ds_season3 = ds_season3[1:]
        seasonmean = ds_season3.groupby('time.year').mean('time') # make annual mean
        # no information the first year of piControl and historical, since we are missing the december month before
        
        # day-weighted rolling 3-months mean for all months (with seasonal variations)
        #day_weighted_avg_allyear = area_avg*day_weights(yrs, chosen_season = 'all')
        #smoothed_allyear = day_weighted_avg_allyear.rolling(min_periods=3, center=True, time=3).mean()
        
        colname = [(exp, member)]
        
        first_member_piControl = 'r1i1p1f1'
        if model in ['CNRM-CM6-1', 'CNRM-ESM2-1', 'UKESM1-0-LL', 'MIROC-ES2L']:
            first_member_piControl = 'r1i1p1f2'
        elif model in ['GISS-E2-1-G']:
            first_member_piControl = 'r101i1p1f1'
        
        if exp == 'piControl' and member == first_member_piControl:
            # create dataframe for storing all results and make the piControl years the index
            df = pd.DataFrame(seasonmean.values, columns = colname)
        else:
            df_col = pd.DataFrame(seasonmean.values, columns = colname)
            df = pd.merge(df, df_col, left_index=True, right_index=True, how='outer')
        
df.columns = pd.MultiIndex.from_tuples(df.columns, names=['Experiment','Member'])


piControl r1i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
years in experiment: 1200      mean weights all 1? True
historical r10i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
years in experiment: 165      mean weights all 1? True
historical r11i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
years in experiment: 165      mean weights all 1? True
historical r1i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
years in experiment: 165      mean weights all 1? True
historical r2i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
years in experiment: 165      mean weights all 1? True
historical r3i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
years in experiment: 165      mean weights all 1? True
historical r4i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
years in experiment: 165      mean weights all 1? True
historical r5i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
years in experiment: 165      mean weights all 1? True
historical r6i1p1f1
Reuse existing fil

In [12]:
# check values in last December for historical
[lastD[member].values for member in lastD.keys()]

[array(303.98845112),
 array(300.70024761),
 array(299.17227101),
 array(297.68419664),
 array(304.08690351),
 array(300.84093148),
 array(302.23400673),
 array(301.63861003),
 array(298.29686613),
 array(300.11572139),
 array(302.87614949)]

## check first and last rows of ssp exps

In [14]:
#pd.set_option('display.min_rows', 90)
df.iloc[0]

Experiment  Member   
piControl   r1i1p1f1            NaN
historical  r10i1p1f1           NaN
            r11i1p1f1           NaN
            r1i1p1f1            NaN
            r2i1p1f1            NaN
            r3i1p1f1            NaN
            r4i1p1f1            NaN
            r5i1p1f1            NaN
            r6i1p1f1            NaN
            r7i1p1f1            NaN
            r8i1p1f1            NaN
            r9i1p1f1            NaN
ssp126      r10i1p1f1    304.114678
            r11i1p1f1    300.894359
            r4i1p1f1     300.938047
ssp245      r10i1p1f1    304.111312
            r11i1p1f1    301.056989
            r4i1p1f1     300.873475
ssp370      r10i1p1f1    304.106286
            r11i1p1f1    300.997615
            r4i1p1f1     300.895037
ssp585      r10i1p1f1    304.062508
            r11i1p1f1    301.004756
            r4i1p1f1     300.910933
Name: 0, dtype: float64

In [15]:
pd.set_option('display.max_columns', 100)
df.iloc[85:88]

Experiment   piControl  historical                                      \
Member        r1i1p1f1   r10i1p1f1   r11i1p1f1    r1i1p1f1    r2i1p1f1   
85          300.141508  300.558808  301.730469  303.499719  297.645194   
86          300.437812  301.353290  297.883701  299.960765  298.699786   
87          298.602817  299.045505  300.841215  300.514987  299.301425   

Experiment                                                              \
Member        r3i1p1f1    r4i1p1f1    r5i1p1f1    r6i1p1f1    r7i1p1f1   
85          302.549496  300.844760  298.027117  302.751669  298.953059   
86          300.088269  301.309380  297.056422  300.569962  299.357938   
87          300.133947  301.704914  299.177810  300.409143  300.821752   

Experiment                              ssp126                         \
Member        r8i1p1f1    r9i1p1f1   r10i1p1f1   r11i1p1f1   r4i1p1f1   
85          301.018605  297.068311  304.124914  301.696208  301.98182   
86          298.803097  297.856097         NaN         NaN        NaN   
87          301.277956  300.256966         NaN         NaN        NaN   

Experiment      ssp245                              ssp370             \
Member       r10i1p1f1   r11i1p1f1    r4i1p1f1   r10i1p1f1  r11i1p1f1   
85          302.884433  304.404938  305.625342  305.091975  306.47196   
86                 NaN         NaN         NaN         NaN        NaN   
87                 NaN         NaN         NaN         NaN        NaN   

Experiment                  ssp585                          
Member        r4i1p1f1   r10i1p1f1   r11i1p1f1    r4i1p1f1  
85          302.885662  306.598372  305.968671  307.307729  
86                 NaN         NaN         NaN         NaN  
87                 NaN         NaN         NaN         NaN

## Save data to file

In [16]:
#df.to_csv('../Processed_data/Nino3_4_DJF/' + model + '_DJF_nino3_4index.txt')

## Similar code as above, but for computing 3-month running mean index for all months:

In [28]:
# For Nino3.4 region:
#latregion = slice(-5,5); lonregion = slice(190, 240) # = 120 W - 170 W
# use larger region before regridding, that adds 5 deg to each border:
#larger_latregion = slice(-10,10); larger_lonregion = slice(185, 245)

# For Nino3 region:
#latregion = slice(-5,5); lonregion = slice(210, 270) # = 150 W - 90 W
#larger_latregion = slice(-10,10); larger_lonregion = slice(205, 275)

# For warm pool:
latregion = slice(-5,5); lonregion = slice(120, 170)
larger_latregion = slice(-10,10); larger_lonregion = slice(115, 175)


resolution = 1;
ds_out = xr.Dataset({'lon': (['lon'], np.arange(lonregion.start+resolution/2, lonregion.stop+resolution/2, resolution)),
                     'lat': (['lat'], np.arange(latregion.start+resolution/2, latregion.stop+resolution/2, resolution))
                    }
                   )
    
regr_lat_bnds = np.array([[upper, upper+resolution] for upper in range(latregion.start,latregion.stop)])
regr_lon_bnds = np.array([[upper, upper+resolution] for upper in range(lonregion.start,lonregion.stop)])
area_w = area_weights(regr_lat_bnds, regr_lon_bnds)

season = 'all'
lastD = {}; lastW = {}

for exp in exp_list:
#for exp in exp_list[:2]:
    key = exp_keys[exp]
    exp_datasets = datasets[key]
    members_sorted = exp_datasets.member_id.sortby(exp_datasets.member_id)
    #for member in [members_sorted.values[0]]: # check for first member only
    for member in members_sorted.values:
        print(exp, member)
        ds = exp_datasets.sel(member_id = member)
        
        # select regional data, perform a regridding, and compute area average
        if model == 'MCM-UA-1-0':
             ds = ds.rename({'longitude': 'lon','latitude': 'lat'}) 
        regional_data = ds.ts.sel(lat = larger_latregion, lon = larger_lonregion)
        regridder = xe.Regridder(regional_data, ds_out, 'bilinear', reuse_weights = True)
        regridded_data = regridder(regional_data)
        area_avg = (regridded_data.transpose('time', 'lon', 'lat') * area_w).mean(dim=['lon', 'lat'])
            
        yrs = int(area_avg.shape[0]/12)
        weights = day_weights(area_avg, chosen_season = season, calendar = ds_calendar)
        
        if exp == 'historical':
            # save last december month for each member for use in season mean in first year of ssp exps
            lastD[member] = area_avg[-1] 
            lastW[member] = weights[-1]
        elif exp not in ['piControl','historical']: # then it must be future scenario   
            area_avg = xr.concat([lastD[member], area_avg], dim = 'time')
            weights = xr.concat([lastW[member], weights], dim = 'time')
            
        # average over season with area weights of mean 1 within each year
        #day_weighted_avg = area_avg*weights
        #ds_season3 = day_weighted_avg.rolling(min_periods=3, center=True, time=3).mean()
        
        # convert to numpy array for increased computational speed
        weights = np.array(weights); area_avg = np.array(area_avg)
        # do rolling mean in for-loop, to give weigths a mean of 1 in each season
        ds_season3 = np.full(len(area_avg), np.nan)
        for t in range(1, len(area_avg)-1):
            season_weigths = weights[t-1:t+2]/weights[t-1:t+2].mean();
            ds_season3[t] = np.mean(area_avg[t-1:t+2]*season_weigths)
        
        if exp not in ['piControl','historical']:
            # remove nan-value obtained from inserting last december month from historical
            ds_season3 = ds_season3[1:]
        
        colname = [(exp, member)]
        
        first_member_piControl = 'r1i1p1f1'
        if model in ['CNRM-CM6-1', 'CNRM-ESM2-1', 'UKESM1-0-LL', 'MIROC-ES2L']:
            first_member_piControl = 'r1i1p1f2'
        elif model in ['GISS-E2-1-G']:
            first_member_piControl = 'r101i1p1f1'
        
        if exp == 'piControl' and member == first_member_piControl:
            # create dataframe for storing all results and make the piControl years the index
            #df = pd.DataFrame(ds_season3.values, columns = colname)
            df = pd.DataFrame(ds_season3, columns = colname)
        else:
            #df_col = pd.DataFrame(ds_season3.values, columns = colname)
            df_col = pd.DataFrame(ds_season3, columns = colname)
            df = pd.merge(df, df_col, left_index=True, right_index=True, how='outer')
        
df.columns = pd.MultiIndex.from_tuples(df.columns, names=['Experiment','Member'])


piControl r1i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
historical r10i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
historical r11i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
historical r1i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
historical r2i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
historical r3i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
historical r4i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
historical r5i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
historical r6i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
historical r7i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
historical r8i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
historical r9i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
ssp126 r10i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
ssp126 r11i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
ssp126 r4i1p1f1
Reuse existing file: bilinear_22x49_10x50.nc
ssp245 r10i1p1f1
Reuse existing fi

In [29]:
df.iloc[:5]

Experiment   piControl  historical                                      \
Member        r1i1p1f1   r10i1p1f1   r11i1p1f1    r1i1p1f1    r2i1p1f1   
0                  NaN         NaN         NaN         NaN         NaN   
1           302.662083  302.507457  302.669390  302.557721  302.826956   
2           302.651524  302.455618  302.715346  302.611521  302.831503   
3           302.815749  302.565136  302.835679  302.780246  302.967399   
4           302.897943  302.699960  302.913178  302.800121  303.128467   

Experiment                                                              \
Member        r3i1p1f1    r4i1p1f1    r5i1p1f1    r6i1p1f1    r7i1p1f1   
0                  NaN         NaN         NaN         NaN         NaN   
1           302.106674  302.361758  302.574408  302.796153  302.676795   
2           302.179875  302.305595  302.510312  302.824386  302.667318   
3           302.372031  302.393990  302.575075  302.983021  302.876859   
4           302.568815  302.618303  302.666980  303.011587  303.074512   

Experiment                              ssp126                          \
Member        r8i1p1f1    r9i1p1f1   r10i1p1f1   r11i1p1f1    r4i1p1f1   
0                  NaN         NaN  302.926720  303.821889  303.792536   
1           302.932538  302.710588  302.738272  303.557922  303.611617   
2           302.818521  302.655856  302.737646  303.499831  303.596628   
3           302.851374  302.773267  302.952857  303.659686  303.677996   
4           302.926157  302.878199  303.157778  303.718754  303.744705   

Experiment      ssp245                              ssp370              \
Member       r10i1p1f1   r11i1p1f1    r4i1p1f1   r10i1p1f1   r11i1p1f1   
0           302.911470  303.817164  303.771626  302.947436  303.850283   
1           302.786166  303.610334  303.564833  302.794248  303.651130   
2           302.885248  303.629159  303.558969  302.825954  303.689122   
3           303.164467  303.806194  303.610733  303.092243  303.870323   
4           303.431848  303.865835  303.761705  303.384038  303.970935   

Experiment                  ssp585                          
Member        r4i1p1f1   r10i1p1f1   r11i1p1f1    r4i1p1f1  
0           303.789172  302.993266  303.818159  303.780367  
1           303.595278  302.868873  303.609931  303.592347  
2           303.651750  302.890026  303.562196  303.615873  
3           303.777566  303.070296  303.736105  303.707214  
4           303.995387  303.283955  303.815404  303.829019

In [30]:
df.iloc[1030:1035]

Experiment   piControl  historical                                      \
Member        r1i1p1f1   r10i1p1f1   r11i1p1f1    r1i1p1f1    r2i1p1f1   
1030        302.918091  303.035058  302.750785  303.362627  302.759335   
1031        302.798370  302.988731  302.681814  303.304790  302.792818   
1032        302.564889  302.890084  302.609276  303.197542  302.740220   
1033        302.349406  302.762958  302.556302  303.157875  302.626402   
1034        302.285530  302.795780  302.625540  303.129683  302.648033   

Experiment                                                              \
Member        r3i1p1f1    r4i1p1f1    r5i1p1f1    r6i1p1f1    r7i1p1f1   
1030        303.232519  303.206122  302.179873  303.181625  302.954050   
1031        303.315409  303.274195  302.143050  303.241229  302.936808   
1032        303.337016  303.251314  301.984532  303.073655  302.893565   
1033        303.285254  303.176620  301.876026  302.775861  302.882195   
1034        303.246477  303.148467  301.949336  302.675178  302.890034   

Experiment                              ssp126                          \
Member        r8i1p1f1    r9i1p1f1   r10i1p1f1   r11i1p1f1    r4i1p1f1   
1030        303.120673  301.762234  304.520877  304.230232  304.324156   
1031        303.071268  301.912529         NaN         NaN         NaN   
1032        302.912777  301.908691         NaN         NaN         NaN   
1033        302.622465  301.887620         NaN         NaN         NaN   
1034        302.534271  302.067474         NaN         NaN         NaN   

Experiment      ssp245                              ssp370              \
Member       r10i1p1f1   r11i1p1f1    r4i1p1f1   r10i1p1f1   r11i1p1f1   
1030        305.118612  305.051343  305.352076  305.655774  306.424606   
1031               NaN         NaN         NaN         NaN         NaN   
1032               NaN         NaN         NaN         NaN         NaN   
1033               NaN         NaN         NaN         NaN         NaN   
1034               NaN         NaN         NaN         NaN         NaN   

Experiment                  ssp585                          
Member        r4i1p1f1   r10i1p1f1   r11i1p1f1    r4i1p1f1  
1030        305.997756  307.638511  307.553447  307.508372  
1031               NaN         NaN         NaN         NaN  
1032               NaN         NaN         NaN         NaN  
1033               NaN         NaN         NaN         NaN  
1034               NaN         NaN         NaN         NaN

## save data to file:

In [32]:
#df.to_csv('../Processed_data/Nino3_4_monthly/' + model + '_nino3_4monthlyindex.txt')
#df.to_csv('../Processed_data/Nino3_monthly/' + model + '_nino3_monthlyindex.txt')
#df.to_csv('../Processed_data/WP_monthly/' + model + '_wp_monthlyindex.txt')